## Getting Started
First generating some test data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from annotation_driven_dataframe_calcs.annotation_driven_dataframe_calcs import expand_for_timesteps
from annotation_driven_dataframe_calcs.column_names import TIMESTEP_NO, ACCOUNT_NO, PARAM_C, PARAM_B, PARAM_A

fake_data = pd.DataFrame({
    ACCOUNT_NO: [1, 2, 3, 4],
    PARAM_A: [201908, 201907, 201906, 201905],
    PARAM_B: [10, 15, 20, 25],
    PARAM_C: [5, 6, 7, 8]
})

In [3]:
result = expand_for_timesteps(fake_data, 1, 3)
result

,ACCT_NO,COLUMN_A,COLUMN_B,COLUMN_C,TIMESTEP_NO
0,1,201908,10,5,1
1,1,201908,10,5,2
2,1,201908,10,5,3
3,2,201907,15,6,1
4,2,201907,15,6,2
5,2,201907,15,6,3
6,3,201906,20,7,1
7,3,201906,20,7,2
8,3,201906,20,7,3
9,4,201905,25,8,1
